In [1]:
import sqlalchemy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

import webbrowser
import plotly.graph_objs as go
import plotly.express as px


In [2]:
cardanoscan_url = 'https://preprod.cardanoscan.io/transaction/'

open_url = 'https://preprod.marlowescan.com/contractView?tab=info&contractId='


In [3]:
engine = sqlalchemy.create_engine('postgresql://jingles:@127.0.0.1:5432/marlowemerged')

In [4]:
df = pd.read_sql('SELECT slotno, txid_hex, datumbytes_hex FROM marlowe.contracts', engine)
df.shape[0]

6014

In [5]:
df = df.drop_duplicates('txid_hex', keep='first')
df.shape[0]

6014

In [6]:
n = 2
def stringToSent(val):
  words = [(val[i:i+n]) for i in range(0, len(val), n)]
  sentence = ' '.join(words)
  return sentence

df['sent'] = df.apply(lambda row: stringToSent(row['datumbytes_hex']),axis=1)
df['length'] = df.apply(lambda row: len(row['sent'].split()),axis=1)

In [7]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['sent'])
print(tfidf_matrix.shape)

(6014, 256)


In [8]:
clf = TruncatedSVD(2)
pca = clf.fit_transform(tfidf_matrix)
print(pca.shape)
pca

(6014, 2)


array([[ 0.75953503, -0.12919283],
       [ 0.91768267, -0.01763109],
       [ 0.79726088, -0.13263937],
       ...,
       [ 0.84337763, -0.09066172],
       [ 0.85142947, -0.24187709],
       [ 0.81591371, -0.12271654]])

In [9]:
df['x'] = pca[:,0]
df['y'] = pca[:,1]

In [10]:
fig = go.FigureWidget([go.Box(y=df.length)])

def do_click(trace, points, state):
    if points.point_inds:
        ind = points.point_inds[0]
        url = df.txid_hex.iloc[ind]
        webbrowser.open_new_tab(open_url+url+'%231')

scatter=fig.data[0]
scatter.on_click(do_click)

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    title="Contracts Sizes",
    xaxis_title="",
    yaxis_title="Size",
)

fig

FigureWidget({
    'data': [{'type': 'box',
              'uid': 'fc9661df-1fc6-443a-9085-1215e870cf50',
              'y': array([1686,  221,  402, ..., 1258,  597, 1634])}],
    'layout': {'autosize': False,
               'height': 800,
               'template': '...',
               'title': {'text': 'Contracts Sizes'},
               'width': 800,
               'xaxis': {'title': {'text': ''}},
               'yaxis': {'title': {'text': 'Size'}}}
})

In [11]:
fig = go.FigureWidget([go.Scatter(x=df.slotno, y=df.length, mode='markers')])

def do_click(trace, points, state):
    if points.point_inds:
        ind = points.point_inds[0]
        url = df.txid_hex.iloc[ind]
        webbrowser.open_new_tab(open_url+url+'%231')

scatter=fig.data[0]
scatter.on_click(do_click)

fig.update_layout(
    autosize=False,
    width=1500,
    height=600,
    title="Contracts over time by size",
    xaxis_title="",
    yaxis_title="",
)

fig

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'uid': '21d80622-7f21-44a3-82c8-89a6ecc6ae50',
              'x': array([20681601, 16637332, 14401271, ..., 30911714, 30654339, 19041486]),
              'y': array([1686,  221,  402, ..., 1258,  597, 1634])}],
    'layout': {'autosize': False,
               'height': 600,
               'template': '...',
               'title': {'text': 'Contracts over time by size'},
               'width': 1500,
               'xaxis': {'title': {'text': ''}},
               'yaxis': {'title': {'text': ''}}}
})

In [12]:
fig = go.FigureWidget([go.Scatter(x=df.x, y=df.y, mode='markers')])

def do_click(trace, points, state):
    if points.point_inds:
        ind = points.point_inds[0]
        url = df.txid_hex.iloc[ind]
        webbrowser.open_new_tab(open_url+url+'%231')

scatter=fig.data[0]
scatter.on_click(do_click)

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    title="Contracts Clusters",
    xaxis_title="",
    yaxis_title="",
)

fig

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'uid': 'eae0706e-5425-4e4d-b892-232fd03dc428',
              'x': array([0.75953503, 0.91768267, 0.79726088, ..., 0.84337763, 0.85142947,
                          0.81591371]),
              'y': array([-0.12919283, -0.01763109, -0.13263937, ..., -0.09066172, -0.24187709,
                          -0.12271654])}],
    'layout': {'autosize': False,
               'height': 800,
               'template': '...',
               'title': {'text': 'Contracts Clusters'},
               'width': 800,
               'xaxis': {'title': {'text': ''}},
               'yaxis': {'title': {'text': ''}}}
})

In [13]:
# df.to_sql(name='contracts_clusters', con=engine, if_exists='replace', schema='marlowe', index=False)

In [14]:
# cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

Index               128
slotno            48112
txid_hex          48112
datumbytes_hex    48112
sent              48112
length            48112
x                 48112
y                 48112
cluster           48112
dtype: int64

In [15]:
from sklearn.cluster import SpectralClustering
import numpy as np


In [16]:
clustering = SpectralClustering(
  n_clusters=6, assign_labels='discretize', random_state=0).fit(tfidf_matrix)
df['cluster'] = clustering.labels_

In [17]:
fig = go.FigureWidget([go.Scatter(
    x=df.x, 
    y=df.y, 
    mode='markers',
    marker={'color': df['cluster'], 'colorscale': 'YlOrRd'},
)])

def do_click(trace, points, state):
    if points.point_inds:
        ind = points.point_inds[0]
        url = df.txid_hex.iloc[ind]
        webbrowser.open_new_tab(open_url+url+'%231')

scatter=fig.data[0]
scatter.on_click(do_click)

fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    title="Contracts Clusters",
    xaxis_title="",
    yaxis_title="",
)

fig

FigureWidget({
    'data': [{'marker': {'color': array([3, 3, 3, ..., 3, 1, 3]),
                         'colorscale': [[0.0, 'rgb(255,255,204)'], [0.125,
                                        'rgb(255,237,160)'], [0.25,
                                        'rgb(254,217,118)'], [0.375,
                                        'rgb(254,178,76)'], [0.5,
                                        'rgb(253,141,60)'], [0.625,
                                        'rgb(252,78,42)'], [0.75,
                                        'rgb(227,26,28)'], [0.875,
                                        'rgb(189,0,38)'], [1.0, 'rgb(128,0,38)']]},
              'mode': 'markers',
              'type': 'scatter',
              'uid': 'a3bbd7e5-b5cc-4dfc-b84e-f921d0ce1a92',
              'x': array([0.75953503, 0.91768267, 0.79726088, ..., 0.84337763, 0.85142947,
                          0.81591371]),
              'y': array([-0.12919283, -0.01763109, -0.13263937, ..., -0.09066172, -0.24187709,
  

In [19]:
# train

In [20]:
print(tfidf_matrix.shape)

(6014, 256)


In [22]:
df_contractslabels = pd.read_sql('SELECT * FROM marlowe.contracts_labels', engine)
df_contractslabels

,id,label
0,80663288260a0c197f2962e0c66c3536e14ac55c41f367...,0
1,e839ffb1bebf8564eca7260fb2d525f0c77034e49cc5cc...,1
2,f9849a6c4182c500d8bf0517ed94671e9f95d6abcf460b...,1
3,72f1f7174a420a12bed5873d0131f8d336315d6a25d92d...,2
4,49ddd677a802474e28bf93f45219f2173933e9f6084d52...,2


In [25]:
df_contractslabels['id'].to_list()

['80663288260a0c197f2962e0c66c3536e14ac55c41f3673d692d278f6eb76cda',
 'e839ffb1bebf8564eca7260fb2d525f0c77034e49cc5cc80a83f7f9b9bf1a3e1',
 'f9849a6c4182c500d8bf0517ed94671e9f95d6abcf460b9052657f5f6ab64493',
 '72f1f7174a420a12bed5873d0131f8d336315d6a25d92d06863531f243d3143c',
 '49ddd677a802474e28bf93f45219f2173933e9f6084d5228e646ea25bc345dfe']

In [23]:
df_contracts = df[['txid_hex', 'Fee']].copy()


,slotno,txid_hex,datumbytes_hex,sent,length,x,y,cluster
0,20681601,000a5348883fb3c0b96936cb7a51787024b9b71b31de80...,d8799fd8799f40ffd8799fa1d8799fd8799fd87980d879...,d8 79 9f d8 79 9f 40 ff d8 79 9f a1 d8 79 9f d...,1686,0.759535,-0.129193,3
1,16637332,000fd04097339261e47db646ca0c597669e215846773ef...,d8799fd8799f40ffd8799fa1d8799fd8799fd87980d879...,d8 79 9f d8 79 9f 40 ff d8 79 9f a1 d8 79 9f d...,221,0.917683,-0.017631,3
2,14401271,0014afa55bcfa78e41c8312a34fab4261c98565a37f1a0...,d8799fd8799f40ffd8799fa1d8799fd8799fd87980d879...,d8 79 9f d8 79 9f 40 ff d8 79 9f a1 d8 79 9f d...,402,0.797261,-0.132639,3
3,30644824,0024f31fea21a4161cdb1d735fc9f11d595435ba2018fb...,d8799fd8799f40ffd8799fa1d8799fd8799fd87980d879...,d8 79 9f d8 79 9f 40 ff d8 79 9f a1 d8 79 9f d...,597,0.853370,-0.177754,1
4,32152686,002b0898ba9ed01da644ca4cdba510106fdb7b92ad5e1a...,d8799fd8799f581c8bb3b343d8e404472337966a722150...,d8 79 9f d8 79 9f 58 1c 8b b3 b3 43 d8 e4 04 4...,318,0.930408,0.283889,0
...,...,...,...,...,...,...,...,...
6009,34551112,ffb5620543c91fee43977df129c6affc1329a3cb83f5a6...,d8799fd8799f40ffd8799fa1d8799fd8799fd87980d879...,d8 79 9f d8 79 9f 40 ff d8 79 9f a1 d8 79 9f d...,283,0.765153,-0.092079,3
6010,16397028,ffc6ba56d15e7cd0f67604da030a54fb26991a63611d92...,d8799fd8799f40ffd8799fa1d8799fd8799fd87980d879...,d8 79 9f d8 79 9f 40 ff d8 79 9f a1 d8 79 9f d...,713,0.806228,-0.064732,4
6011,30911714,ffccac42ba58c88fd5e96c93b97789902cb0028517a08f...,d8799fd8799f40ffd8799fa1d8799fd8799fd87980d879...,d8 79 9f d8 79 9f 40 ff d8 79 9f a1 d8 79 9f d...,1258,0.843378,-0.090662,3
6012,30654339,ffe7b10f8bbb4f6a5bba759fa0d2c091c904356de5b3eb...,d8799fd8799f40ffd8799fa1d8799fd8799fd87980d879...,d8 79 9f d8 79 9f 40 ff d8 79 9f a1 d8 79 9f d...,597,0.851429,-0.241877,1
